In [1]:
import pandas as pd
train_df = pd.read_table('1. 실습용자료.txt',sep='|',encoding="cp949")
test_df = pd.read_table('2. 모델개발용자료.txt',sep='|',encoding="cp949")

In [2]:
train_df.to_csv("1. 실습용자료.csv",encoding="utf-8-sig",index=False)
test_df.to_csv("2. 모델개발용자료.csv",encoding="utf-8-sig",index=False)

In [3]:
train_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매


In [4]:
test_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영


- text_obj, text_mthd, text_deal 3가지 텍스트 데이터를 기반으로 산업분류코드 예측하기

### 탐색적 데이터 분석(EDA)
- digit_1 : 대분류 21개
- digit_2 : 소분류 232개
- digit_3 : 세분류 495개
- '한국표준산업분류(10차)_국문.csv'의 표준산업분류 코드별 항목명 존재

실습용자료 불러오기, 열이름 수정, 열삭제

In [5]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#train_df.drop("AI_id",axis=1,inplace=True)

결측치 확인

In [6]:
train_df[train_df["text_obj"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
231,id_0000232,G,46,465,NaN,"초음파세척기, 음식물처리기 도매업",NaN
305,id_0000306,Q,87,872,NaN,재가요양서비스기관,NaN
310,id_0000311,I,55,551,NaN,NaN,펜션운영업
353,id_0000354,G,46,464,NaN,향초 도매,NaN
...,...,...,...,...,...,...,...
999765,id_0999766,G,47,472,NaN,건강분말 구매후 판매,NaN
999800,id_0999801,C,22,222,NaN,"제품입고,숏트, 철,플라스틱",NaN
999848,id_0999849,F,42,424,NaN,토탈인테리어및 유리바닥재도소매,NaN
999913,id_0999914,J,59,592,NaN,녹음시설운영업,NaN


In [7]:
train_df[train_df["text_mthd"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
21,id_0000022,R,91,911,당구,NaN,NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
58,id_0000059,G,46,467,"지붕,강판,집수리, 인테리어,태양광구조물",NaN,NaN
...,...,...,...,...,...,...,...
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN
999987,id_0999988,I,55,551,펜션운영업,NaN,NaN


In [8]:
train_df[train_df["text_deal"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
10,id_0000011,C,10,102,멸치,"입고, 가공",NaN
21,id_0000022,R,91,911,당구,NaN,NaN
29,id_0000030,C,29,292,Resim,"재료구입, 세미앗세이",NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
...,...,...,...,...,...,...,...
999954,id_0999955,I,56,561,한식 음식점,NaN,NaN
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN


중복값 확인 : https://rfriend.tistory.com/266
- 대/중/소분류가 일치하더라도 text의 차이가 존재한다
- duplicated_df : 대/중/소분류에 중복이 존재하는 row
- 거의 모든 데이터가 대/중/소분류에 중복이 존재함
- 999064번 데이터 같은 경우는 text_obj과 text_mthd가 바꿔서 기입된 것이 아닌가 하는 의문

In [9]:
duplicated_df = train_df[train_df.duplicated(["대분류","중분류","소분류"])].sort_values(["대분류","중분류","소분류"])
duplicated_df

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
5242,id_0005243,A,1,11,벼,모판생산,농가 출하
9375,id_0009376,A,1,11,사업장에서,오이.고추를재배,오이.고추를판매
16182,id_0016183,A,1,11,벼씨,발아,묘종
20631,id_0020632,A,1,11,NaN,NaN,벼재배
20987,id_0020988,A,1,11,벼,재배하여,벼재배
...,...,...,...,...,...,...,...
998903,id_0998904,S,96,969,영업장에서,일반인을대상으로,고속화물 수하물보관소
999001,id_0999002,S,96,969,세탁소에서,고객의의뢰를받다,"세탁,드라이크리닝"
999064,id_0999065,S,96,969,사망자대상으로,화장터에서,"화장,묘지분양관리"
999794,id_0999795,S,96,969,영업장에서,일반인을대상으로,고양이카페


표준사업분류 코드별 항목명 매핑해주기
- 앞 행의 데이터로 결측치 채우기 : https://seong6496.tistory.com/84
- 특정 열의 데이터 타입 변경하기 : https://hogni.tistory.com/70

In [10]:
code_df = pd.read_csv("한국표준산업분류(10차)_국문.csv")
code_df = code_df.fillna(method="ffill")[["대분류","대분류_항목명",
                                          "중분류","중분류_항목명",
                                         "소분류","소분류_항목명",
                                         "세분류","세분류_항목명",
                                         "세세분류","세세분류_항목명"]]
code_df = code_df.astype({'중분류': 'int',
                          '소분류': 'int',
                         '세분류': 'int',
                          '세세분류': 'int'})


중복값 확인
- 세분류의 111번과 소분류의 111번은 아예 다른 표준산업분류임

In [11]:
code_df[code_df["소분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
89,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11111,탁주 및 약주 제조업
90,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11112,맥아 및 맥주 제조업
91,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11119,기타 발효주 제조업


In [12]:
code_df[code_df["세분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
0,A,"농업, 임업 및 어업(01~03)",1,농업,11,작물 재배업,111,곡물 및 기타 식량작물 재배업,1110,곡물 및 기타 식량작물 재배업


표준산업분류와 실습자료 연결
- 대분류/중분류/소분류_항목명 연결

In [124]:
code_df = code_df[["대분류","대분류_항목명","중분류","중분류_항목명","소분류","소분류_항목명"]]
code_df = code_df.drop_duplicates(['대분류','중분류','소분류'], keep='first')

merge_df = pd.merge(train_df,code_df,on=["대분류","중분류","소분류"],how="inner")
#merge_df.to_csv("실습용자료_한국표준산업분류.csv",encoding="utf-8-sig",index=False)

In [125]:
merge_df.head(3)

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,대분류_항목명,중분류_항목명,소분류_항목명
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업
1,id_0000017,S,95,952,사업장에서,고객요청으로,자동차세차업,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업
2,id_0000110,S,95,952,자동차,전문적으로 수리,자동차 전문수리업,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",개인 및 소비용품 수리업,자동차 및 모터사이클 수리업


null 값 채우기

In [126]:
merge_df.fillna("",inplace=True)

한글 이외의 문자 처리
- 쉼표 -> 공백으로 변경하기
- 다중공백 -> 단일공백으로 변경하기 : https://m.blog.naver.com/wideeyed/221906671758

In [127]:
import re
def data_cleansing_text(df,col_lst):
    for col in col_lst:
        df[col] = df[col].apply(lambda x : re.sub(',', ' ', x))
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
    return df

In [128]:
merge_df = data_cleansing_text(merge_df,["text_obj","text_mthd","text_deal"])

정규식을 이용한 한글 분류
- re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') : 영어 및 숫자,기호 제외
- re.compile('[\u3131-\u3163\uac00-\ud7a3]+') : 영어 및 숫자,기호 등
- re.sub 로도 한글, 영어 분류가능
- null값(np.NaN)

In [129]:
# 한글만 추출하는 함수
def hangul_only(text):
    hangul = re.compile('([^ \u3131-\u3163\uac00-\ud7a3]+)')
    result = hangul.sub("",text)
    return result

# 한글만 제외하는 함수
def hangul_except(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    result = hangul.sub("",text)
    return result

In [130]:
# 3개 열에 대해서 각각 한글만 있는, 한글이 없는 열만 생성하기
df = merge_df
col_lst = ["text_obj","text_mthd","text_deal"]

for col in  col_lst:
    df[col+"_kr"] = df[col].apply(lambda x : hangul_only(x).strip()) # 한글만 있는 열 + 양쪽 공백제거
    df[col+"_nokr"] = df[col].apply(lambda x : hangul_except(x).strip()) # 한글이 없는 열 + 양쪽 공백제거

In [138]:
merge_df[["text_obj_kr","text_obj_nokr","text_obj"]].tail(5)

,text_obj_kr,text_obj_nokr,text_obj
999995,금속 테잎용원단,,금속 테잎용원단
999996,무연탄,,무연탄
999997,잎담배 판상엽,,잎담배 판상엽
999998,향료,,향료
999999,담배잎 담배잎줄기,,담배잎 담배잎줄기


한글 제외한 열 확인해보기
- 공백("")을 Null값으로 변경해서 공백만 존재하는 행 제외하고 확인하기
- 단순히 한글과 그 이외의 것으로 분류하면 안된다
- 자동차정비(3급) -> 자동차정비급 으로 분류된다
- 서비스 ( 자동차 -> 기록 시에 오류난 것

In [173]:
merge_df[merge_df["text_obj_nokr"] != ""].to_csv("실습용자료_nokr.csv",encoding="utf-8-sig",index=False)
# 한글이 제외된 텍스트를 열에 담아놓은 데이터프레임

특정 문자열만 포함한 행 출력하기 : https://zephyrus1111.tistory.com/46
- '㎡' 포함한 행만 출력한 결과, 대부분 G 47에 속함, 매장의 면적을 명시함

In [169]:
merge_df[merge_df["text_obj"].str.contains("㎡")][["대분류","중분류","소분류"]].value_counts()
                                                 #"대분류_항목명","중분류_항목명","소분류_항목명"]].value_counts()

대분류  중분류  소분류
G    47   471    3010
          472      20
          474       4
          476       4
          475       3
          478       3
I    56   561       2
G    46   463       1
N    75   752       1
R    91   912       1
S    96   961       1
dtype: int64

hanspell 맞춤법 검사기 이해
- 맞춤법 검사 후, 맞춤법 혹은 표준어가 의심되는 단어나 구절 key,value로 반환

In [268]:
from hanspell import spell_checker
from hanspell.constants import CheckResult

In [260]:
text = "안녕 하세요 저는 한국인 임니다 이 문장은 한ㅇ입니다"
result = spell_checker.check(text)

for key,value in result.words.items():
    print(key,value)

안녕하세요 2
저는 0
한국인입니다 1
이 0
문장은 0
한ㅇ입니다 0


## 전처리 진행
- 문자 사이에 끼인 기호 처리하기, 정규표현식 이해 : https://data-newbie.tistory.com/210

In [269]:
import re
def data_cleansing_text(df,col_lst):
    for col in col_lst:
        # null값 채우기
        df[col] = df[col].fillna("")
        
        # 양쪽 공백 삭제
        df[col] = df[col].apply(lambda x : x.strip())
        
        # 영어 대문자 -> 소문자
        df[col] = df[col].apply(lambda x : x.lower())
        
        # 영어 문자 사이의 특수기호 삭제
        df[col] = df[col].str.replace(pat="(?<=[a-z])[/$](?=[a-z])",repl = "",regex=True)
        
        # 특수기호 -> 공백으로
        df[col] = df[col].apply(lambda x : re.sub('[^\w\s]', ' ', x))
        
        # 다중공백 -> 단일공백
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
        # 한글 자음모음 삭제
        df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
        
        # ㎡ 변환
        df[col] = df[col].apply(lambda x : re.sub('㎡',"제곱미터",x))
        
        # 맞춤법 검사기
        df[col] = df[col].apply(lambda x : spell_checker.check(x).checked)
    return df

In [ ]:
# train_df에 대해서 데이터 전처리
train_df_cleans =  data_cleansing_text(train_df,["text_obj","text_mthd","text_deal"])

In [267]:
train_df

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소 과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [ ]:
text = "일반고객을대상으로"
result = spell_checker.check(text)

for key,value in result.words.items():
    print(key,value)